In [1]:
from backtesting.test import GOOG

GOOG.tail()

/Users/liupeng/PycharmProjects/ZBot/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/liupeng/PycharmProjects/ZBot/.venv/lib/python3.13/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

,Open,High,Low,Close,Volume
2013-02-25,802.3,808.41,790.49,790.77,2303900
2013-02-26,795.0,795.95,784.40,790.13,2202500
2013-02-27,794.8,804.75,791.11,799.78,2026100
2013-02-28,801.1,806.99,801.03,801.20,2265800
2013-03-01,797.8,807.14,796.15,806.19,2175400


In [2]:
import pandas as pd


def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

In [10]:
from backtesting import Strategy
from backtesting.lib import crossover


class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        print(f"Close {self.data.Close[-1]} index:{self.data.index[-1]}")
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

In [11]:
from backtesting import Backtest

bt = Backtest(GOOG, SmaCross, cash=10_000, commission=.002)
stats = bt.run()
stats

Close 117.49 index:2004-09-17 00:00:00
Close 119.36 index:2004-09-20 00:00:00
Close 117.84 index:2004-09-21 00:00:00
Close 118.38 index:2004-09-22 00:00:00
Close 120.82 index:2004-09-23 00:00:00
Close 119.83 index:2004-09-24 00:00:00
Close 118.26 index:2004-09-27 00:00:00
Close 126.86 index:2004-09-28 00:00:00
Close 131.08 index:2004-09-29 00:00:00
Close 129.6 index:2004-09-30 00:00:00
Close 132.58 index:2004-10-01 00:00:00
Close 135.06 index:2004-10-04 00:00:00
Close 138.37 index:2004-10-05 00:00:00
Close 137.08 index:2004-10-06 00:00:00
Close 138.85 index:2004-10-07 00:00:00
Close 137.73 index:2004-10-08 00:00:00
Close 135.26 index:2004-10-11 00:00:00
Close 137.4 index:2004-10-12 00:00:00
Close 140.9 index:2004-10-13 00:00:00
Close 142.0 index:2004-10-14 00:00:00
Close 144.11 index:2004-10-15 00:00:00
Close 149.16 index:2004-10-18 00:00:00
Close 147.94 index:2004-10-19 00:00:00
Close 140.49 index:2004-10-20 00:00:00
Close 149.38 index:2004-10-21 00:00:00
Close 172.43 index:2004-10-22

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                    94.27374
Equity Final [$]                  56263.51934
Equity Peak [$]                   56309.05934
Commissions [$]                   10563.95154
Return [%]                          462.63519
Buy & Hold Return [%]               607.37036
Return (Ann.) [%]                    22.46598
Volatility (Ann.) [%]                 37.4129
CAGR [%]                             14.99343
Sharpe Ratio                          0.60049
Sortino Ratio                          1.1445
Calmar Ratio                           0.6621
Alpha [%]                           450.62135
Beta                                  0.01978
Max. Drawdown [%]                   -33.93159
Avg. Drawdown [%]                    -6.16072
Max. Drawdown Duration      830 days 00:00:00
Avg. Drawdown Duration       50 days 00:00:00
# Trades                          